Copyright 2019 [BIG CHENG](mailto:name@email.com)
Licensed under the Apache License, Version 2.0 (the "License");

#Deep learning based facial emotion recognition and emotion drivened reinforcement learning agent
Lots of people living in modern world feel unhappy. Can you image, "Globally, more than 300 million people of all ages suffer from depression." (from [WHO/depression](https://www.who.int/news-room/fact-sheets/detail/depression))

#Install Tensorflow 2.0

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

import tensorflow as tf

print(tf.__version__)

#Import Packages

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display


#Import Dataset


In [0]:
## using google drive
from google.colab import drive
drive.mount('/content/gdrive')
#Copy dataset here ...
#One simple option is Kaggle/FER2013 

#inspect some data
!ls Training | head
display(Image('Training/i00000_e0_t0.png'))
display(Image('Training/i00002_e2_t0.png'))

#Preprocess Data

In [0]:
IMAGE_SIZE = 96
BATCH_SIZE = 32

def fn_mapping(fname, label):
  image_decoded = tf.image.decode_jpeg(tf.io.read_file(fname))
  image_normalized = (tf.cast(image_decoded, tf.float32)/127.5) - 1
  image_resized = tf.image.resize(image_normalized, (IMAGE_SIZE, IMAGE_SIZE))
  image_concated = tf.concat([image_resized, image_resized, image_resized], 2)
  return image_concated, label

#fnames_train = [os.path.join(dir_name, fname) for fname in filenames]
#lables_train = ...
#fnames_val = [os.path.join(dir_name, fname) for fname in filenames]
#lables_val = ...
data2train = tf.data.Dataset.from_tensor_slices((tf.constant(fnames_train), tf.constant(labels_train))).map(fn_mapping).shuffle(buffer_size=10000).batch(BATCH_SIZE)
data2val = tf.data.Dataset.from_tensor_slices((tf.constant(fnames_val), tf.constant(labels_val))).map(fn_mapping).batch(BATCH_SIZE)

#Define Model

In [0]:
# Base model - MobileNetV2
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')
maxpool_layer = tf.keras.layers.GlobalMaxPooling2D()
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

model = tf.keras.Sequential([
    base_model,
    maxpool_layer,
    prediction_layer
])

learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.summary()

#Train Model

In [0]:
num_epochs = 10
steps_per_epoch = round(len(labels_train))//BATCH_SIZE
validation_steps = 20

history = model.fit(data2train.repeat(),
                    epochs=num_epochs,
                    steps_per_epoch = steps_per_epoch,
                    validation_data=data2val.repeat(), 
                    validation_steps=validation_steps)

#Recognize Emotion

In [0]:
# image from camera, preprocessed and resized to IMAGE_SIZE
# imgs2pred = np.zeros((num_pred, IMAGE_SIZE, IMAGE_SIZE, 3))
# ...

cls = model.predict(imgs2pred)

for i in range(num_pred):
  print (cls[i])